## creating a n-dimensional hypersphere 

encodes the RGB information of each pixel of an image.

In [1]:
from PIL import Image
from qiskit.quantum_info import Statevector

import numpy as np
from scipy.interpolate import Rbf

coverting image to n-dimensional unit statevectors on an n-dimensional bloch sphere

In [2]:
image_path = "/Users/devaldeliwala/quantum_image_encryption/images/el_primo_square.jpg"
n =8

im = Image.open(image_path, 'r')
image = im.convert("RGB")

n_pixels = np.array(image).shape[0]**2
pixel_values = np.array(list(image.getdata()))

def rgb_to_decimal(r, g, b):
    return (r << 16) + (g << 8) + b

def decimal_to_rgb(decimal):
    r = (decimal >> 16) & 0xFF
    g = (decimal >> 8) & 0xFF
    b = decimal & 0xFF
    return r, g, b

int_values = []
for r,g,b in pixel_values: 
    int_values.append(rgb_to_decimal(r, g, b))
int_values = np.array(int_values)

recovered_rgb_values = []
for i in int_values: 
    recovered_rgb_values.append(decimal_to_rgb(i))
recovered_rgb_values = np.array(recovered_rgb_values)

# checking for reversibility
if(np.allclose(pixel_values, recovered_rgb_values)): 
   print("rgb-int reversible conversion success")

rgb-int reversible conversion success


In [3]:
groups = []

i = 0 
while i < n_pixels: 
    groups.append(int_values[i:i + 2 ** n])
    i += 2**n

def to_unit_vector(arr):
    magnitude = np.linalg.norm(arr)
    if magnitude == 0:
        return arr, magnitude
    unit_vector = arr / magnitude
    return unit_vector, magnitude

def from_unit_vector(unit_vector, magnitude):
    return unit_vector * magnitude

unit_vector_groups = []
magnitudes = []

# unit_vector_groups stores 2^n-dimensional unit vectors 
for i in groups: 
    unit_vector, magnitude = to_unit_vector(i)
    magnitudes.append(magnitude)
    unit_vector_groups.append(unit_vector)

In [4]:
pixel_values[:, 0][0]

19

In [5]:
rbf_r_list = []
rbf_g_list = []
rbf_b_list = []

r_values = [pixel[0] for pixel in pixel_values]
g_values = [pixel[1] for pixel in pixel_values]
b_values = [pixel[2] for pixel in pixel_values]

# Setup RBF for each dimension separately
for i in range(256):
    rbf_r = Rbf(*np.array(unit_vector_groups).T, r_values[i::256], function='multiquadric')
    rbf_g = Rbf(*np.array(unit_vector_groups).T, g_values[i::256], function='multiquadric')
    rbf_b = Rbf(*np.array(unit_vector_groups).T, b_values[i::256], function='multiquadric')
    rbf_r_list.append(rbf_r)
    rbf_g_list.append(rbf_g)
    rbf_b_list.append(rbf_b)

recovered_pixel_values = []
for j in unit_vector_groups: 
    r_values = [rbf_r_list[i](*j) for i in range(256)]
    g_values = [rbf_g_list[i](*j) for i in range(256)]
    b_values = [rbf_b_list[i](*j) for i in range(256)]

    rgb_values = np.array([np.clip([r, g, b], 0, 255).astype(int) for r, g, b in zip(r_values, g_values, b_values)])
    recovered_pixel_values.append(rgb_values)

recovered_pixel_values

[array([[ 19, 102, 234],
        [ 19, 103, 235],
        [ 18, 102, 235],
        [ 17, 101, 234],
        [ 16,  99, 233],
        [ 15, 100, 234],
        [ 12,  99, 236],
        [ 12,  99, 236],
        [ 16, 102, 241],
        [ 16, 101, 241],
        [ 16, 102, 244],
        [ 18, 103, 245],
        [ 18, 103, 247],
        [ 19, 104, 248],
        [ 21, 106, 250],
        [ 21, 106, 251],
        [ 20, 106, 254],
        [ 18, 109, 252],
        [ 16, 111, 250],
        [ 13, 114, 249],
        [ 13, 114, 252],
        [ 17, 113, 252],
        [ 25, 110, 253],
        [ 29, 110, 248],
        [ 39, 117, 253],
        [ 32, 116, 250],
        [ 24, 115, 255],
        [ 17, 114, 255],
        [ 19, 111, 255],
        [ 22, 112, 255],
        [ 25, 115, 255],
        [ 28, 118, 253],
        [ 20, 114, 250],
        [ 22, 117, 253],
        [ 28, 118, 255],
        [ 27, 118, 254],
        [ 23, 115, 255],
        [ 20, 113, 254],
        [ 17, 112, 255],
        [ 14, 114, 255],


In [6]:
final_pixels = []
for i in recovered_pixel_values: 
    for j in i: 
        final_pixels.append(j)
final_pixels = np.array(final_pixels).reshape(64, 64, 3)

In [7]:
image = Image.fromarray(np.uint8(final_pixels), 'RGB')
image.show()

In [21]:
# converting each of the 2^n dimensional unit vectors reversibly into 
# n dimensional statevectors 

def unit_vector_to_statevector(unit_vector):
    dim = len(unit_vector) // 2
    real_parts = unit_vector[:dim]
    imag_parts = unit_vector[dim:]
    statevector = real_parts + 1j * imag_parts
    statevector /= np.linalg.norm(statevector)
    return statevector

def statevector_to_unit_vector(statevector):
    real_parts = np.real(statevector)
    imag_parts = np.imag(statevector)
    combined = np.concatenate([real_parts, imag_parts])
    unit_vector = combined / np.linalg.norm(combined)
    return unit_vector

statevectors = []
for i in unit_vector_groups: 
    statevectors.append(unit_vector_to_statevector(i))

unit_vectors = []
for i in statevectors:
    unit_vectors.append(statevector_to_unit_vector(i))
    
statevectors = [Statevector(i) for i in statevectors]

for i in statevectors:
    display(i.draw('latex'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

converting the n-dimensional statevectors back into the RGB values that encode the image

In [9]:
def statevector_to_unit_vector(statevector):
    real_parts = np.real(statevector)
    imag_parts = np.imag(statevector)
    combined = np.concatenate([real_parts, imag_parts])
    unit_vector = combined / np.linalg.norm(combined)
    return unit_vector

recovered_unit_vectors = []

for i in statevectors:
    recovered_unit_vectors.append(statevector_to_unit_vector(i.data))


int_vectors = []
for i in range(len(recovered_unit_vectors)): 
    int_vectors.append(recovered_unit_vectors[i] * magnitudes[i])

recovered_int_values = []
for i in int_vectors: 
    for j in i: 
        recovered_int_values.append(j)
    
recovered_int_values = np.array(recovered_int_values)

recovered_pixel_values = []
for i in recovered_int_values: 
    recovered_pixel_values.append(decimal_to_rgb(int(i)))
recovered_pixel_values = np.array(recovered_rgb_values)


In [15]:
width, height = image.size

reshaped_pixel_values = recovered_pixel_values.reshape((height, width ,3))
reconstructed_image = Image.fromarray(reshaped_pixel_values.astype(np.uint8), 'RGB')
reconstructed_image.show()

In [18]:
recovered_unit_vectors

[array([0.0508522 , 0.0508522 , 0.04822156, 0.04559083, 0.04296011,
        0.04296011, 0.03508862, 0.03508862, 0.04298086, 0.04298086,
        0.04560143, 0.04823215, 0.05086291, 0.05349364, 0.05612444,
        0.05613472, 0.05613484, 0.04829385, 0.04308358, 0.03787336,
        0.03786324, 0.04834507, 0.06927797, 0.07974932, 0.10340515,
        0.08768239, 0.0640883 , 0.04573494, 0.05094512, 0.05880646,
        0.06931896, 0.07459052, 0.05621656, 0.05886784, 0.07460079,
        0.07197011, 0.06407806, 0.05619625, 0.04833491, 0.04048381,
        0.04311449, 0.03786332, 0.04046326, 0.04829373, 0.06139584,
        0.06923655, 0.0822978 , 0.0770567 , 0.06661574, 0.04562154,
        0.03769923, 0.05342198, 0.09534861, 0.10581996, 0.09268718,
        0.06122127, 0.04560147, 0.01672478, 0.03771862, 0.03504643,
        0.09018916, 0.04019516, 0.04546688, 0.06388194, 0.04296003,
        0.04296003, 0.04559083, 0.04559083, 0.04559083, 0.04559087,
        0.04035006, 0.04298078, 0.04824231, 0.04

In [22]:
def unit_vector_to_statevector(unit_vector):

            dim = len(unit_vector) // 2
            real_parts = unit_vector[:dim]
            imag_parts = unit_vector[dim:]
            statevector = real_parts + 1j * imag_parts
            statevector /= np.linalg.norm(statevector)

            return statevector

statevectors_recovered = []

for i in range(len(recovered_unit_vectors)): 
    statevectors_recovered.append(unit_vector_to_statevector(recovered_unit_vectors[i]))

statevectors_recovered = [Statevector(i) for i in statevectors_recovered]

for i in statevectors_recovered: 
        display(i.draw('latex'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [27]:
np.random.rand(3, 2)

array([[0.32131272, 0.23973794],
       [0.76240934, 0.83850467],
       [0.80348154, 0.62702362]])